In [1]:
# Cell 1: Silence noisy logs & install dependencies
import os
import warnings
import logging

# 1) Silence TF/TPU logs
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
logging.getLogger().setLevel(logging.ERROR)

# 2) Suppress tqdm “IProgress not found” warning by matching its text
warnings.filterwarnings("ignore", message="IProgress not found")

# 3) Install required packages
!pip install --upgrade pip setuptools wheel
!pip install --quiet transformers torch torchvision pillow scikit-learn pandas opencv-python-headless

print("✅ Cell 1 completed: dependencies installed and warnings silenced")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.3 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-gbq 0.28.0 requires google-api-core<3.0.0dev,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 86.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 45.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Cell 2: Load & sample the Kaggle dataset (auto-detect folder)

import os, glob, random, pandas as pd

# 1. Search /content and /content/data for a folder that has train/, validation/, test/
BASE = None
for parent in ['/content','/kaggle/input/deepfake-and-real-images']:
    if not os.path.isdir(parent): 
        continue
    for d in os.listdir(parent):
        p = os.path.join(parent, d)
        if not os.path.isdir(p):
            continue
        subs = [name.lower() for name in os.listdir(p)]
        if all(x in subs for x in ['train','validation','test']):
            BASE = p
            break
    if BASE:
        break

if BASE is None:
    raise FileNotFoundError(
        "Could not auto-detect the dataset folder under /content or /content/data. "
        f"Found: {os.listdir('/content')} and {os.listdir('/content/data') if os.path.isdir('/content/data') else 'N/A'}"
    )
print(f"▶️ Found dataset at: {BASE}")

# 2. Gather image paths from train/validation/test splits (real & fake)
paths = []
for split in os.listdir(BASE):
    if split.lower() not in ('train','validation','test'):
        continue
    split_dir = os.path.join(BASE, split)
    for label in os.listdir(split_dir):
        if label.lower() not in ('real','fake'):
            continue
        paths += glob.glob(os.path.join(split_dir, label, '*'))

if not paths:
    raise FileNotFoundError(f"No images found under detected BASE: {BASE}")

# 3. Sample 100 images
random.seed(42)
sample = random.sample(paths, min(100, len(paths)))
df = pd.DataFrame({
    'image': sample,
    'true':  [os.path.basename(os.path.dirname(p)).upper() for p in sample]
})

print(f"✅ Cell 2 completed: loaded and sampled {len(df)} images from {BASE}")


▶️ Found dataset at: /kaggle/input/deepfake-and-real-images/Dataset
✅ Cell 2 completed: loaded and sampled 100 images from /kaggle/input/deepfake-and-real-images/Dataset


In [5]:
# Cell 3: CPU-only inference, ensemble & binary metrics with 10 HF models

import os
# Force CPU/PyTorch, disable TF/TPU
os.environ['USE_TF'] = '0'
os.environ['CUDA_VISIBLE_DEVICES'] = ''

from transformers import pipeline
from collections import Counter
from sklearn.metrics import confusion_matrix, precision_score, f1_score, accuracy_score
import pandas as pd

# 10 Hugging Face deepfake-vs-real detectors
MODEL_IDS = [
    "dima806/deepfake_vs_real_image_detection",
    "prithivMLmods/Deep-Fake-Detector-Model",
    "HrutikAdsare/deepfake-detector-faceforensics",
    "prithivMLmods/AI-vs-Deepfake-vs-Real-v2.0",
    "ashish-001/deepfake-detection-using-ViT",
    "prithivMLmods/Deepfake-Detect-Siglip2",
    "Wvolf/ViT_Deepfake_Detection",
    "dima806/ai_vs_real_image_detection",
    "joyc360/deepfakes",
    "Skullly/DeepFake-image-detection-ViT-384"
]

# Load pipelines (skip any that fail)
pipes = {}
for m in MODEL_IDS:
    try:
        pipes[m] = pipeline(
            "image-classification",
            model=m,
            trust_remote_code=True,
            framework="pt",
            device=-1
        )
        print(f"✅ Loaded {m}")
    except Exception as e:
        print(f"⚠️ Skipped {m}: {type(e).__name__}")
assert pipes, "No models loaded successfully."

# Inference & majority-vote ensemble
preds = []
for img in df['image']:
    votes = []
    for p in pipes.values():
        try:
            votes.append(p(img)[0]['label'].upper())
        except:
            pass
    preds.append(Counter(votes).most_common(1)[0][0] if votes else 'REAL')
df['pred'] = preds

# Convert to binary 1=REAL, 0=FAKE
y_true = [1 if t == 'REAL' else 0 for t in df['true']]
y_pred = [1 if p == 'REAL' else 0 for p in df['pred']]

# Compute metrics
tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
metrics = {
    'Sensitivity (Recall)': tp/(tp+fn) if (tp+fn) else 0,
    'Specificity':           tn/(tn+fp) if (tn+fp) else 0,
    'Precision':             precision_score(y_true, y_pred, zero_division=0),
    'F1-Score':              f1_score(y_true, y_pred, zero_division=0),
    'Accuracy':              accuracy_score(y_true, y_pred)
}

# Display results
print("\n## Ensemble Results (100-sample test)\n")
print(pd.DataFrame.from_dict(metrics, orient='index', columns=['Value']).to_markdown())
print("\n✅ Cell 3 completed")


Device set to use cpu
Device set to use cpu


✅ Loaded dima806/deepfake_vs_real_image_detection
✅ Loaded prithivMLmods/Deep-Fake-Detector-Model


Device set to use cpu


✅ Loaded HrutikAdsare/deepfake-detector-faceforensics


Device set to use cpu


...

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/344M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

Device set to use cpu


✅ Loaded Skullly/DeepFake-image-detection-ViT-384

## Ensemble Results (100-sample test)

|                      |    Value |
|:---------------------|---------:|
| Sensitivity (Recall) | 1        |
| Specificity          | 0.716981 |
| Precision            | 0.758065 |
| F1-Score             | 0.862385 |
| Accuracy             | 0.85     |

✅ Cell 3 completed
